# 1. Entendimiento del Negocio
## Objetivo de Negocio
El principal objetivo del negocio, es aumentar las ganancias por ventas en un 10% para el año 2026, en contraste con lo vendido en 2025.

## Problema
El sistema actual muestra artículos aleatorios en un carrusel. Se necesita diseñar e implementar un Sistema Recomendador que sugiera ítems de interés para cada usuario, reemplazando la lógica aleatoria por una predictiva. Esto busca mejorar la experiencia del usuario y, consecuentemente, aumentar la tasa de conversión y el volumen de ventas, alineándose con el objetivo de negocio.

## Consideraciones

* **Items Fijos:** El comitente solo vende 100 artículos y no se agregarán ítems nuevos ("por cábala siempre vende 100").Por lo que la cantidad de items es fija.
* **Usuarios Nuevos:** La plataforma recibe "muchos usuarios nuevos todo el tiempo", lo que obliga a tener una estrategia robusta de recomendación inicial (Cold Start).
* **Volumen de Compra:** Los usuarios compran en promedio 7 u 8 artículos, a lo mucho 10.
* **Ausencia de Datos:** No existe una base de datos preexistente, por lo que se debe diseñar e implementar una para almacenar usuarios, ítems y preferencias.
* **Items Genericos:** El comitente no nos especifica los items, los deja a nuestra interpretación, siempre que el sistema funciona

# 2.Entendimiento de los datos
## Datos a Almacenar
Dado que no hay una base de datos existente, debemos diseñar una que modele las entidades y relaciones del sistema.
Debemos almacenar:
* **Usuarios:** En la documentación de la API suministrada por el comitente se espeficifica que debe tener los atributos:
    * *id* que es el identificador de tipo *integer*
    * *username* que es el nombre de usuario de tipo *string*
    `ejemplo: pepitaLaPistolera`
    * *attributes* de tipo *object* que almacena atributos de forma dinamica.
    `ejemplo: {
        'campo1': 'valor1',
        'campo2': 'valor2'
        }`
* **Items:** En la documentación suministrada por el comitente se espeficifica que siempre habra exactamente 100 items y que cada uno debe tener los atributos:
    * *id* que es el identificador de tipo *integer*
    * *name* que es el nombre del item de tipo *string*
    `ejemplo: Las mil y una noches`
    * *attributes* de tipo *object* que almacena atributos de forma dinamica.
    `ejemplo: {
        'campo1': 'valor1',
        'campo2': 'valor2'
        }`
* **Preferencias de los usuarios por los items:** El comitente nos deja este apartado a nuestro parecer. Diseñamos almacerlas en una entidad llamada "Preferences", que registra el rating que le da un usuario a un item que compro", que almacene los atributos:
    * *user_id* que es el identificador del usuario de tipo *integer*
    * *item_id* que es el identificador del item de tipo *integer*
    * *preference_value*: la ponderación (rating) que le da un usuario al item que compro. Es de tipo *integer* y tiene un valor entre 1 y 5. Luego de comprar un item, el usuario lo reseña indicando que le parecio el item a traves de estrellas, como lo hacen plataformas parecias como Mercado Libre.

## Implementación de la persistencia de datos
Se diseñó una base de datos SQLite (*recommendation_system.db*) para almacenar las tres entidades principales del sistema.
Las razones principales por la que elegimos SQLite en lugar sobre otras opciones son su ligereza, portabilidad y facilidad de uso, ya que no requiere un servidor, se ejecuta como una biblioteca en el mismo proceso de la aplicación, almacena la base de datos en un único archivo, y es ideal para prototipos o para ejecutar localmente sin conexión a internet.

### Codigo SQL para la creacion de las tablas
**Tabla users**
 ```SQL
 CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY NOT NULL,
                username TEXT UNIQUE NOT NULL,
                attributes TEXT
            );
 ```
**Tabla items**
 ```SQL
CREATE TABLE IF NOT EXISTS items (
                    item_id INTEGER PRIMARY KEY NOT NULL,
                    name TEXT NOT NULL,
                    attributes TEXT
                );
 ```
 **Tabla preferences**
 ```SQL
 CREATE TABLE IF NOT EXISTS preferences (
                user_id INTEGER NOT NULL,
                item_id INTEGER NOT NULL,
                preference_value INTEGER NOT NULL,
                PRIMARY KEY (user_id, item_id),
                FOREIGN KEY(user_id) REFERENCES users(id) ON DELETE CASCADE,
                FOREIGN KEY(item_id) REFERENCES items(item_id) ON DELETE CASCADE
            );

 ```
### Prevención ante valores perdidos
Prevenimos valores perdidos mediante al momento de crear la base de datos utilizano los constraint:
* **NOT NULL**: Es para evitar que datos importantes tomen el valor *null*.
* **ON DELETE CASCADE**: Es para no tener preferencias con referencias nulas que generen inconsistencias en el modelo. Al eliminarse un usuario, se eliminan todas las preferencias de este.

### Generación de los datos iniciales
Generamos un dataset inicial para demostración de la aplicación respetando las consideraciones expresadas por el comitente. 
Utilizamos la integencia artificial Gemini para generar 700 Usuarios, 100 Items y aproximadamente 5600 Preferencias. Estos datos fueron almacenados en formato *.csv* en los archivos:
* **Items.csv:** El archivo tiene el formato interno
```csv
item_id,name,price,category
1,Crónicas de la Sombra,15.99,Ficción Fantástica
```
* **Preferences.csv:** El archivo tiene el formato interno
```csv
user_id,item_id,preference_value,interaction_date
1,55,4,12/1/2025
```
* **Users.csv:** El archivo tiene el formato interno
```csv
item_id,name,price,category
1,Crónicas de la Sombra,15.99,Ficción Fantástica
```



# 3. Preparación de los Datos


Cargamos los datos desde los CSVs usando la libreria Pandas

In [249]:
import pandas as pd
USERS_URL = './Datasets/Users.csv'# Ruta local del archivo CSV de usuarios
ITEMS_URL = './Datasets/Items.csv'# Ruta local del archivo CSV de items
PREFERENCES_URL = './Datasets/Preferences.csv'# Ruta local del archivo CSV de preferencias
users_df = pd.read_csv(USERS_URL)
items_df = pd.read_csv(ITEMS_URL)
preferences_df = pd.read_csv(PREFERENCES_URL)

Preprocesamos los atributos de USERS

In [250]:
import json
if 'id' not in users_df.columns: 
    id_cols = [col for col in users_df.columns if 'id' in col.lower()]
    if id_cols:
        users_df = users_df.rename(columns={id_cols[0]: 'id'})

BASE_KEYS = ['id', 'username']
# Función para serializar atributos adicionales a JSON
def serialize_attributes(row):
    attributes = {
        k: v for k, v in row.items() 
        if k not in BASE_KEYS and pd.notna(v)
    }
    return json.dumps(attributes)# Serializamos a JSON

users_df['attributes'] = users_df.apply(serialize_attributes, axis=1)# Creamos la columna 'attributes' con JSON
users_df_clean = users_df[BASE_KEYS + ['attributes']].copy()# Filtramos solo las columnas necesarias        

Preprocesamiento de ITEMS

In [251]:

if 'id' in items_df.columns and 'item_id' not in items_df.columns:
    items_df.rename(columns={'id': 'item_id'}, inplace=True)
    
BASE_ITEM_KEYS = ['item_id', 'name']
def serialize_item_attributes(row):
    attributes = {
        k: v for k, v in row.items() 
        if k not in BASE_ITEM_KEYS and pd.notna(v)
    }
    return json.dumps(attributes)

items_df['attributes'] = items_df.apply(serialize_item_attributes, axis=1)
items_df_clean = items_df[BASE_ITEM_KEYS + ['attributes']].copy()

Preprocesamiento de PREFERENCES 

In [252]:
# Mapeo y Filtrado de PREFERENCES 
# Aseguramos que las columnas tengan los nombres correctos
preferences_df.rename(columns={
    'user_id': 'user_id',
    'item_id': 'item_id',
    'preference_value': 'preference_value'
}, inplace=True)

# Filtramos solo las columnas necesarias 
preferences_df = preferences_df[['user_id', 'item_id', 'preference_value']].copy()

# Aseguramos que ITEMS tenga 'item_id'
# Asumiendo que el CSV de Items usa 'item_id'
if 'id' in items_df.columns and 'item_id' not in items_df.columns:
    items_df.rename(columns={'id': 'item_id'}, inplace=True)
    

Una vez preprocesados los csv, creamos la base de datos con el formato que se indicado en la fase anterior

In [253]:
import sqlite3
DB_NAME = 'recommendation_system.db'# Nombre del archivo de la base de datos SQLite
try:
    conn = sqlite3.connect(DB_NAME)# Conexión a la base de datos SQLite
    cursor = conn.cursor()# Cursor para ejecutar comandos SQL
    
    #Creamos tabla de usuarios con id como PRIMARY KEY y username único
    cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY NOT NULL,
            username TEXT UNIQUE NOT NULL,
            attributes TEXT
        );
    """)
    
    # Creamos la tabla de items con item_id como PRIMARY KEY
    cursor.execute("""
            CREATE TABLE IF NOT EXISTS items (
                item_id INTEGER PRIMARY KEY NOT NULL,
                name TEXT NOT NULL,
                attributes TEXT
            );
        """)
    
    #Creamos la tabla de preferencias con claves foráneas a las tablas users e items y clave primaria compuesta
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS preferences (
            user_id INTEGER NOT NULL,
            item_id INTEGER NOT NULL,
            preference_value INTEGER NOT NULL,
            PRIMARY KEY (user_id, item_id),
            FOREIGN KEY(user_id) REFERENCES users(id) ON DELETE CASCADE,
            FOREIGN KEY(item_id) REFERENCES items(item_id) ON DELETE CASCADE
        );
    """)
    
    #  Deshabilitamos la verificación de FK temporalmente (solo para carga inicial) para evitar errores de inserción
    conn.execute("PRAGMA foreign_keys = OFF;") 

    # Insertamos los datos en las tablas correspondientes
    users_df_clean.to_sql('users', conn, if_exists='append', index=False)
    items_df_clean.to_sql('items', conn, if_exists='append', index=False) 
    preferences_df.to_sql('preferences', conn, if_exists='append', index=False)
    
    # Volvemos a habilitar la verificación de claves foráneas
    conn.execute("PRAGMA foreign_keys = ON;")
    
    conn.commit()# Guardamos los cambios
    conn.close()# Cerramos la conexión
    
    print(f" Base de datos SQLite creada y cargada con éxito.")
        
except Exception as e:
    print(f" Error al inicializar SQLite: {e}")
    raise

 Error al inicializar SQLite: UNIQUE constraint failed: users.id


IntegrityError: UNIQUE constraint failed: users.id

Verificamos que exista la base de datos

In [ ]:

import os
if os.path.exists(DB_NAME): # Verificamos si la base de datos ya existe
    print(f"Base de datos '{DB_NAME}' encontrada.")
else:
    print(f"Base de datos '{DB_NAME}' no encontrada.")

Base de datos 'recommendation_system.db' encontrada.


Una vez lista la base de datos obtemos los datos desde la misma para crear las estructuras necesarias para el modelo

Creamos un dataframe principal que contenga id_usuario, id_item, nombre del item y ranking que le dio el usuario

In [ ]:
#Consulta SQL para unir items y preferences
SQL_QUERY = """
SELECT 
    T1.item_id,
    T1.name,
    T2.user_id,
    T2.preference_value
FROM 
    items AS T1 
INNER JOIN 
    preferences AS T2
ON 
    T1.item_id = T2.item_id;
"""
    
conn = sqlite3.connect(DB_NAME)# Abrimos la conexión a la base de datos
df = pd.read_sql_query(SQL_QUERY, conn)# Leemos las preferencias uniendo items y preferences
df.head()# Mostramos las primeras filas del DataFrame resultante

,item_id,name,user_id,preference_value
0,55,Meteoritos y Cometas,1,4
1,82,Egipto Faraónico,1,4
2,10,Biografía de Marie Curie,1,2
3,33,Gestión de Proyectos PMP,1,5
4,91,Robots en el Tiempo,1,3


El dataframe users_df tiene la información de los usuarios

In [ ]:
users_df = pd.read_sql_query("SELECT id, username, attributes FROM users", conn)# Leemos los usuarios desde la base de datos
users_df.head()# Mostramos las primeras filas del DataFrame de usuarios

,id,username,attributes
0,1,jgonzalez01@mail.com,"{""telephone"": ""11-4501-1001"", ""birthdate"": ""5/..."
1,2,mrodriguez02@mail.com,"{""telephone"": ""11-4501-1002"", ""birthdate"": ""11..."
2,3,pperez03@mail.com,"{""telephone"": ""11-4501-1003"", ""birthdate"": ""3/..."
3,4,llopez04@mail.com,"{""telephone"": ""11-4501-1004"", ""birthdate"": ""7/..."
4,5,acarcia05@mail.com,"{""telephone"": ""11-4501-1005"", ""birthdate"": ""9/..."


El dataframe users_df tiene la información de los usuarios

In [ ]:

items_df = pd.read_sql_query("SELECT * FROM items", conn)# Leemos los items desde la base de datos
items_df.head()# Mostramos las primeras filas del DataFrame de items

,item_id,name,attributes
0,1,Crónicas de la Sombra,"{""price"": 15.99, ""category"": ""Ficci\u00f3n Fan..."
1,2,El Jardín Silencioso,"{""price"": 22.5, ""category"": ""Novela Hist\u00f3..."
2,3,Guía de Inversión Inteligente,"{""price"": 35.0, ""category"": ""Econom\u00eda y F..."
3,4,Recetas de la Abuela,"{""price"": 18.75, ""category"": ""Cocina""}"
4,5,Atlas Mundial Actualizado,"{""price"": 49.99, ""category"": ""Geograf\u00eda""}"


In [ ]:
conn.close()# Cerramos la conexión a la base de datos

Deserialización de los atributos del usuario

In [ ]:
if 'attributes' in users_df.columns:
        users_df['attributes'] = users_df['attributes'].apply(
            lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else {}
        )
users_df.head()
        

,id,username,attributes
0,1,jgonzalez01@mail.com,"{'telephone': '11-4501-1001', 'birthdate': '5/..."
1,2,mrodriguez02@mail.com,"{'telephone': '11-4501-1002', 'birthdate': '11..."
2,3,pperez03@mail.com,"{'telephone': '11-4501-1003', 'birthdate': '3/..."
3,4,llopez04@mail.com,"{'telephone': '11-4501-1004', 'birthdate': '7/..."
4,5,acarcia05@mail.com,"{'telephone': '11-4501-1005', 'birthdate': '9/..."


Deserialización de los atributos del item

In [ ]:
# Deserialización de los atributos del item
if 'attributes' in items_df.columns:
    # 1. Deserializar el JSON y convertirlo en una Serie de diccionarios
    items_df['attributes_dict'] = items_df['attributes'].apply(
        lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else {}
    )
    # Expenimos los diccionarios en columnas separadas
    temp_attr_df = items_df['attributes_dict'].apply(pd.Series)
    # Concatenamos las columnas expandidas al DataFrame principal y renombrar para evitar colisión
    # El DataFrame ITEMS_DF final tendrá 'item_id', 'name', 'attributes' (el JSON original), y las columnas expandidas.
    # Ahora el DF contiene todos los atributos como columnas separadas.
    items_df = pd.concat([items_df, temp_attr_df], axis=1)
if 'id' in users_df.columns: #nos aseguramos que el id sea int
    users_df['id'] = users_df['id'].astype(int)
items_df.head()

,item_id,name,attributes,attributes_dict,price,category
0,1,Crónicas de la Sombra,"{""price"": 15.99, ""category"": ""Ficci\u00f3n Fan...","{'price': 15.99, 'category': 'Ficción Fantásti...",15.99,Ficción Fantástica
1,2,El Jardín Silencioso,"{""price"": 22.5, ""category"": ""Novela Hist\u00f3...","{'price': 22.5, 'category': 'Novela Histórica'}",22.50,Novela Histórica
2,3,Guía de Inversión Inteligente,"{""price"": 35.0, ""category"": ""Econom\u00eda y F...","{'price': 35.0, 'category': 'Economía y Finanz...",35.00,Economía y Finanzas
3,4,Recetas de la Abuela,"{""price"": 18.75, ""category"": ""Cocina""}","{'price': 18.75, 'category': 'Cocina'}",18.75,Cocina
4,5,Atlas Mundial Actualizado,"{""price"": 49.99, ""category"": ""Geograf\u00eda""}","{'price': 49.99, 'category': 'Geografía'}",49.99,Geografía


# 4.Modelado
## Selección de técnica de modelado
Dado que en el planteamiento del problema se nos especifica que la cantidad de ítems es fija, el numero de usuarios aumenta y registramos las preferencias del usuario, se seleccionó construir un sistema recomendador de filtro colaborativo, basado en usuarios.
Utilizaremos la similitud entre usuarios en función de sus preferencias para realizar recomendaciones. Recordemos que como hipótesis tenemos que usuarios similares tienden a gustarle items similares.
Los pasos a seguir son los siguientes:

1. Encontrar usuarios similares en función de los ratings que han dado a los productos de la base de datos.
2. Identificar los items que usuarios similares han ranqueado alto, y los usuarios no han consumido
3. Recomendar items en función de este ranking.

## Construcción del modelo
### Matriz de preferencias
Vamos a crear un amatriz Usuario x Item con el puntaje dado por cada usuario a cada item que compro. Es una matriz dispersa por la que habrá muchos valores nulos.

In [ ]:
matrix = df.pivot_table(index='user_id', columns='name', values='preference_value')# Creamos la matriz usuario-item
matrix.head()

name,Agujeros Negros,Algoritmos Avanzados en Java,Aromaterapia,Asesinato en el Tren Nocturno,Atlas Mundial Actualizado,Bases de Datos SQL,Biografía de Marie Curie,Bolsa de Valores Avanzada,Caligrafía Japonesa,Cartas de Amor Perdidas,...,Sin Rastro,Sonetos al Mar,Sueños de Androides,Tapas Españolas,Testigo Ocular,Versos Olvidados,Voces de la Guerra Civil,Yoga Aéreo,Yoga para Principiantes,Época Medieval
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN
3,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


Hay personas que tienden a dar un puntaje mayor que otras, por lo que normalizaremos los valores. 
Hacemos que todos los puntajes tengan la misma media (0) y la misma desviación estandar (1)
Luego de la normalización, los valores negativos corresponden a las puntuaciones que un usuario da por debajo de su promedio.

In [ ]:

user_item_matrix = matrix.copy()# hacemos una copia para trabajar sin alterar la original
row_mean = user_item_matrix.mean(axis=1)# Calculamos el promedio por fila
row_std = user_item_matrix.std(axis=1)# Calculamos la desviación estándar por fila
row_std[row_std == 0] = 1 # Evitamos división por cero
matrix_norm = user_item_matrix.sub(row_mean, axis=0).div(row_std, axis=0)# Normalizamos la matriz usuario-item
matrix_norm.head()

name,Agujeros Negros,Algoritmos Avanzados en Java,Aromaterapia,Asesinato en el Tren Nocturno,Atlas Mundial Actualizado,Bases de Datos SQL,Biografía de Marie Curie,Bolsa de Valores Avanzada,Caligrafía Japonesa,Cartas de Amor Perdidas,...,Sin Rastro,Sonetos al Mar,Sueños de Androides,Tapas Españolas,Testigo Ocular,Versos Olvidados,Voces de la Guerra Civil,Yoga Aéreo,Yoga para Principiantes,Época Medieval
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,-1.106797,NaN,NaN,NaN,...,0.474342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,0.380319,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.253546,NaN,NaN,1.014185,NaN,NaN,1.014185,NaN
3,NaN,NaN,NaN,NaN,-0.283193,NaN,NaN,NaN,-1.227168,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.587011,-0.083527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.918796,NaN,NaN,0.751742,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.490788


### Similitud entre usuarios
Usaremos la similitud de coseno para determinar la similitud entre usuarios.
Esto mide el "ángulo" entre los vectores de preferencias de dos usuarios en el espacio de ítems. Un valor cercano a 1 indica alta similitud.

Rellenamos los valores NaN con 0 para calcular similitudes

In [ ]:
user_item_matrix_filled = matrix_norm.fillna(0)# Rellenamos los valores NaN con 0
user_item_matrix_filled.head()

name,Agujeros Negros,Algoritmos Avanzados en Java,Aromaterapia,Asesinato en el Tren Nocturno,Atlas Mundial Actualizado,Bases de Datos SQL,Biografía de Marie Curie,Bolsa de Valores Avanzada,Caligrafía Japonesa,Cartas de Amor Perdidas,...,Sin Rastro,Sonetos al Mar,Sueños de Androides,Tapas Españolas,Testigo Ocular,Versos Olvidados,Voces de la Guerra Civil,Yoga Aéreo,Yoga para Principiantes,Época Medieval
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,-1.106797,0.0,0.000000,0.0,...,0.474342,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.380319,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,-0.253546,0.0,0.000000,1.014185,0.0,0.000000,1.014185,0.000000
3,0.0,0.000000,0.000000,0.000000,-0.283193,0.0,0.000000,0.0,-1.227168,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.0,1.587011,-0.083527,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,-0.918796,0.000000,0.0,0.751742,0.000000,0.000000
5,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,-1.490788


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
user_similarity_cosine = cosine_similarity(user_item_matrix_filled)# Calculamos la similitud del coseno entre los usuarios
user_item_matrix_filled.head()

name,Agujeros Negros,Algoritmos Avanzados en Java,Aromaterapia,Asesinato en el Tren Nocturno,Atlas Mundial Actualizado,Bases de Datos SQL,Biografía de Marie Curie,Bolsa de Valores Avanzada,Caligrafía Japonesa,Cartas de Amor Perdidas,...,Sin Rastro,Sonetos al Mar,Sueños de Androides,Tapas Españolas,Testigo Ocular,Versos Olvidados,Voces de la Guerra Civil,Yoga Aéreo,Yoga para Principiantes,Época Medieval
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,-1.106797,0.0,0.000000,0.0,...,0.474342,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.380319,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,-0.253546,0.0,0.000000,1.014185,0.0,0.000000,1.014185,0.000000
3,0.0,0.000000,0.000000,0.000000,-0.283193,0.0,0.000000,0.0,-1.227168,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.0,1.587011,-0.083527,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,-0.918796,0.000000,0.0,0.751742,0.000000,0.000000
5,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,-1.490788


Los valores estan en el intervalo [-1, 1], donde valores positivos significan mayor similitud.

### Matriz de similitud entre usuarios

In [ ]:
user_ids = user_item_matrix_filled.index.tolist()# Obtenemos los IDs de usuario
user_similarity = pd.DataFrame( #Creamos la matriz de similitud de usuarios
    user_similarity_cosine, 
    index=user_ids, 
    columns=user_ids
)
user_similarity.index = user_similarity.index.astype(str)# Aseguramos que los índices sean strings
user_similarity.columns = user_similarity.columns.astype(str)# Aseguramos que las columnas sean strings
user_similarity.head()

,1,2,3,4,5,6,7,8,9,10,...,691,692,693,694,695,696,697,698,699,700
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.026530,-0.017687,-0.160947,-0.045985,0.183940,0.201626,0.000000,0.000000,0.000000,-0.074283
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029778,...,-0.034034,0.000000,0.134717,0.008508,0.000000,0.000000,-0.124791,0.000000,0.106356,0.000000
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.260813,0.000000,-0.140438,0.000000,-0.015839,-0.041181,0.000000,0.000000,0.133046,0.000000
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.014715,...,0.000000,-0.012146,0.000000,-0.311131,-0.227976,0.000000,0.000000,-0.293379,0.000000,0.000000
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.286501,...,-0.039416,0.030320,0.000000,-0.018192,0.000000,-0.150083,-0.259234,0.016676,0.183434,0.030320


Dado un usuario

In [ ]:

user = "350"

Veamos los items comprados por el mismo

In [ ]:
user_items = matrix_norm.loc[int(user)]# Obtenemos las preferencias del usuario
items_comprados = user_items[user_items.notna()].index.tolist()# Items comprados por el usuario
items_comprados

['Desarrollo Web con React',
 'El Legado de Einstein',
 'Fotografía Digital',
 'Geología Terrestre',
 'Masajes Terapéuticos',
 'Mitología Griega',
 'Robots en el Tiempo']

Obtenemos los 30 usuarios mas similares a 

In [ ]:
n=30
        
similar_users = ( # Obtenemos usuarios similares
        user_similarity[user]
        .sort_values(ascending=False)
        .drop(user, errors='ignore')
        .head(n)
    )
similar_users

322    1.000000
378    1.000000
308    1.000000
336    1.000000
364    1.000000
392    1.000000
449    0.312799
421    0.312799
237    0.312799
209    0.312799
265    0.312799
293    0.312799
539    0.208418
519    0.208418
499    0.208418
549    0.208418
479    0.208418
469    0.208418
459    0.208418
529    0.208418
489    0.208418
509    0.208418
180    0.198507
442    0.148810
230    0.148810
202    0.148810
414    0.148810
258    0.148810
286    0.148810
647    0.108740
Name: 350, dtype: float64

Veamos que items compraron usuarios similares

In [ ]:
# items comprados por usuarios similares
#  Obtenemos la lista de IDs de usuarios similares asegurandonos que sean enteros
similar_users_ids = similar_users.index.astype(int).tolist()
print(f"Usuarios similares a {user}: {similar_users_ids}")

## Filtrar matrix_norm para obtener solo las filas de los usuarios similares
# Usamos .loc[] para indexar por las etiquetas del índice (los user_id).
similar_user_items = matrix_norm.loc[similar_users_ids]
# Identificar los ítems que *al menos un* usuario similar ha comprado/calificado
items_comprados_usuarios_similares = similar_user_items.columns[
    similar_user_items.notna().any()
].tolist()

items_comprados_usuarios_similares

Usuarios similares a 350: [322, 378, 308, 336, 364, 392, 449, 421, 237, 209, 265, 293, 539, 519, 499, 549, 479, 469, 459, 529, 489, 509, 180, 442, 230, 202, 414, 258, 286, 647]


['Asesinato en el Tren Nocturno',
 'Ciberseguridad Esencial',
 'Civilizaciones Antiguas',
 'Cloud Computing AWS',
 'Criptomonedas y Blockchain',
 'Crónicas de la Sombra',
 'Desarrollo Web con React',
 'Desarrollo de Videojuegos Unity',
 'El Legado de Einstein',
 'El Silencio del Testigo',
 'Escultura Moderna',
 'Finanzas Personales 2026',
 'Fotografía Digital',
 'Geología Terrestre',
 'Historia del Arte Renacentista',
 'La Balada del Viajero',
 'La Conquista del Espacio',
 'La Profecía del Sol',
 'La Revolución Industrial',
 'La Vida en Marte',
 'Masajes Terapéuticos',
 'Microeconomía Aplicada',
 'Mitología Griega',
 'Panadería Artesanal',
 'Pilates en Casa',
 'Recetas de la Abuela',
 'Repostería Francesa',
 'Robots en el Tiempo',
 'Sonetos al Mar',
 'Yoga Aéreo',
 'Época Medieval']

Items candidatos para recomendar (Son los items comprados por usuarios similares excluyendo los que el usuario ya compro)

In [ ]:
candidate_items = list(set(items_comprados_usuarios_similares) - set(items_comprados))
candidate_items

['La Vida en Marte',
 'Yoga Aéreo',
 'La Balada del Viajero',
 'Recetas de la Abuela',
 'Microeconomía Aplicada',
 'La Conquista del Espacio',
 'Repostería Francesa',
 'El Silencio del Testigo',
 'Desarrollo de Videojuegos Unity',
 'Sonetos al Mar',
 'La Profecía del Sol',
 'Crónicas de la Sombra',
 'Ciberseguridad Esencial',
 'La Revolución Industrial',
 'Civilizaciones Antiguas',
 'Escultura Moderna',
 'Finanzas Personales 2026',
 'Cloud Computing AWS',
 'Asesinato en el Tren Nocturno',
 'Época Medieval',
 'Panadería Artesanal',
 'Historia del Arte Renacentista',
 'Criptomonedas y Blockchain',
 'Pilates en Casa']

Vamos a ponderar los items de forma tal que la puntuación sea igual al promedio ponderado entre la similitud del usuario y el rating que le dió ese usuario.
Los usuarios más similares van a pesar más en el rating.

In [ ]:
similar_user_preferences = matrix_norm.loc[similar_users_ids]# Preferencias de usuarios similares
items_comprados_por_similares = similar_user_preferences.columns[similar_user_preferences.notna().any()].tolist()# Items comprados por usuarios similares
candidate_items = list(set(items_comprados_por_similares) - set(items_comprados))# Items candidatos para recomendar
candidate_matrix= similar_user_preferences[candidate_items].fillna(0).copy()# Rellenamos NaN con 0
similar_users.index = similar_users.index.astype(candidate_matrix.index.dtype)# Aseguramos que los índices coincidan
weighted_scores = candidate_matrix.multiply(similar_users, axis=0)# Puntuaciones ponderadas
sum_similarity = similar_users.sum()# Suma de similitudes
recommendation_scores = weighted_scores.sum(axis=0) / sum_similarity# Puntuaciones finales
recommendation_scores

name
La Vida en Marte                  -0.027174
Yoga Aéreo                         0.030236
La Balada del Viajero             -0.003397
Recetas de la Abuela               0.083149
Microeconomía Aplicada            -0.015285
La Conquista del Espacio           0.083149
Repostería Francesa               -0.315010
El Silencio del Testigo            0.020380
Desarrollo de Videojuegos Unity   -0.128504
Sonetos al Mar                    -0.052334
La Profecía del Sol                0.094143
Crónicas de la Sombra              0.030236
Ciberseguridad Esencial            0.259150
La Revolución Industrial          -0.032115
Civilizaciones Antiguas           -0.032115
Escultura Moderna                  0.004325
Finanzas Personales 2026          -0.022677
Cloud Computing AWS                0.219858
Asesinato en el Tren Nocturno     -0.003397
Época Medieval                     0.192688
Panadería Artesanal               -0.256917
Historia del Arte Renacentista     0.008492
Criptomonedas y Blockchain 

Ordenamos las recomendaciones de forma descendente segun su puntuacion

In [ ]:
recomendaciones_ordenadas = recommendation_scores.sort_values(ascending=False)# Ordenamos las recomendaciones

#### Recomendaciones

In [ ]:
recomendaciones_finales = recomendaciones_ordenadas.index.tolist()# Lista final de recomendaciones
print(f"Items recomendados para el usuario {user}: ")
recomendaciones_finales

Items recomendados para el usuario 350: 


['Ciberseguridad Esencial',
 'Cloud Computing AWS',
 'Criptomonedas y Blockchain',
 'Época Medieval',
 'La Profecía del Sol',
 'Recetas de la Abuela',
 'La Conquista del Espacio',
 'Yoga Aéreo',
 'Crónicas de la Sombra',
 'El Silencio del Testigo',
 'Pilates en Casa',
 'Historia del Arte Renacentista',
 'Escultura Moderna',
 'Asesinato en el Tren Nocturno',
 'La Balada del Viajero',
 'Microeconomía Aplicada',
 'Finanzas Personales 2026',
 'La Vida en Marte',
 'Civilizaciones Antiguas',
 'La Revolución Industrial',
 'Sonetos al Mar',
 'Desarrollo de Videojuegos Unity',
 'Panadería Artesanal',
 'Repostería Francesa']

### Correccion del cold start
El problema del Cold Start se presenta cuando un usuario es nuevo en la plataforma y, por lo tanto, no ha interactuado con ningún ítem. Sin historial de preferencias, el algoritmo de Filtro Colaborativo Basado en Usuarios no puede calcular la similitud con otros usuarios, lo que resulta en la imposibilidad de generar recomendaciones.
Como solucion el sistema aplica una estrategia de recomendación basada en popularidad.
La popularidad de un ítem se mide por el número de veces que ha sido calificado en el historial de preferencias.
Se agrupan las preferencias po ítem y se cuenta el número total de registros para cada uno.
Esta estrategia es eficaz porque asegura que el usuario nuevo sea expuesto a los artículos que han tenido mayor éxito entre la base de usuarios existente, maximizando la probabilidad de una interacción inicial positiva. Una vez que el usuario nuevo realiza una primera calificación, su ID ingresa a la matriz de preferencias y el sistema pasa a usar el modelo principal en futuras solicitudes.

#### Top 10 items mas comprados

In [ ]:
n=10
top_items = df.groupby('name')['preference_value'].count().sort_values(ascending=False)# Contamos la cantidad de preferencias por item y los ordenamos
top_items.head(n).index.tolist()# Retornamos los nombres de los items más populares

['Fitness Funcional',
 'Mitología Griega',
 'El Último Dragón',
 'Crimen en la Mansión',
 'Biografía de Marie Curie',
 'Historia del Arte Renacentista',
 'El Misterio de la Calle Nueve',
 'La Ciudad bajo la Niebla',
 'Haikus Japoneses',
 'Ciberseguridad Esencial']

#### Activacion del Cold Start
Se activa en dos escenarios dentro de la función de recomendación principal
1. Usuario sin preferencias: para esto definimos la siguiente funcion

In [ ]:
def user_has_preferences(user_id: int) -> bool:
    """Verifica si un usuario tiene preferencias registradas en la matriz normalizada.
    Args:
        user_id (int): ID del usuario a verificar.
    Returns:
        bool: True si el usuario tiene preferencias, False en caso contrario."""
    return user_id in matrix_norm.index
# Ejemplo de uso
user_id = 350 #ya sabemos que este usuario tiene preferencias
if user_has_preferences(user_id):
    print(f"El usuario {user_id} tiene preferencias registradas.")
user_id = 9999 #suponemos que este usuario no existe
if not user_has_preferences(user_id):
    print(f"El usuario {user_id} no tiene preferencias registradas.")

El usuario 350 tiene preferencias registradas.
El usuario 9999 no tiene preferencias registradas.


2. Fallo del modelo principal: El modelo intenta la predicción, pero la suma de similitudes es cero.

## Evaluacion del modelo
Primero encapsulamos el sistema en funciones para simplicidad


In [278]:
def initialize_db():
    """
    Crea y puebla la base de datos SOLO si el archivo DB no existe.
    Establece las claves primarias (compuestas) y foráneas.
    """
    try:
        # Cargamos los datos desde los CSV
        users_df = pd.read_csv(USERS_URL)
        items_df = pd.read_csv(ITEMS_URL)
        preferences_df = pd.read_csv(PREFERENCES_URL)
        
        # Preprocesamos los atributos de USERS
        if 'id' not in users_df.columns: 
            id_cols = [col for col in users_df.columns if 'id' in col.lower()]
            if id_cols:
                users_df = users_df.rename(columns={id_cols[0]: 'id'})
        
        BASE_KEYS = ['id', 'username']
        # Función para serializar atributos adicionales a JSON
        def serialize_attributes(row):
            attributes = {
                k: v for k, v in row.items() 
                if k not in BASE_KEYS and pd.notna(v)
            }
            return json.dumps(attributes)# Serializamos a JSON
        
        
        users_df['attributes'] = users_df.apply(serialize_attributes, axis=1)# Creamos la columna 'attributes' con JSON
        users_df_clean = users_df[BASE_KEYS + ['attributes']].copy()# Filtramos solo las columnas necesarias        
        
        # Preprocesamiento de ITEMS 
        if 'id' in items_df.columns and 'item_id' not in items_df.columns:
            items_df.rename(columns={'id': 'item_id'}, inplace=True)
            
        BASE_ITEM_KEYS = ['item_id', 'name']
        def serialize_item_attributes(row):
            attributes = {
                k: v for k, v in row.items() 
                if k not in BASE_ITEM_KEYS and pd.notna(v)
            }
            return json.dumps(attributes)
        
        items_df['attributes'] = items_df.apply(serialize_item_attributes, axis=1)
        items_df_clean = items_df[BASE_ITEM_KEYS + ['attributes']].copy()

        # Mapeo y Filtrado de PREFERENCES 
        # Aseguramos que las columnas tengan los nombres correctos
        preferences_df.rename(columns={
            'user_id': 'user_id',
            'item_id': 'item_id',
            'preference_value': 'preference_value'
        }, inplace=True)
        
        # Filtramos solo las columnas necesarias 
        preferences_df = preferences_df[['user_id', 'item_id', 'preference_value']].copy()
        
        # Aseguramos que ITEMS tenga 'item_id'
        # Asumiendo que el CSV de Items usa 'item_id'
        if 'id' in items_df.columns and 'item_id' not in items_df.columns:
            items_df.rename(columns={'id': 'item_id'}, inplace=True)
            
        # ------------------------------------------------------------------------------------------------
        # Creación de la base de datos SQLite y tablas con claves primarias y foráneas
        # ------------------------------------------------------------------------------------------------
        conn = sqlite3.connect(DB_NAME)# Conexión a la base de datos SQLite
        cursor = conn.cursor()# Cursor para ejecutar comandos SQL
        
       #Creamos tabla de usuarios con id como PRIMARY KEY y username único
        cursor.execute("""
             CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY NOT NULL,
                username TEXT UNIQUE NOT NULL,
                attributes TEXT
            );
        """)
        
        # Creamos la tabla de items con item_id como PRIMARY KEY
        cursor.execute("""
                CREATE TABLE IF NOT EXISTS items (
                    item_id INTEGER PRIMARY KEY NOT NULL,
                    name TEXT NOT NULL,
                    attributes TEXT
                );
            """)
        
        #Creamos la tabla de preferencias con claves foráneas a las tablas users e items y clave primaria compuesta
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS preferences (
                user_id INTEGER NOT NULL,
                item_id INTEGER NOT NULL,
                preference_value INTEGER NOT NULL,
                PRIMARY KEY (user_id, item_id),
                FOREIGN KEY(user_id) REFERENCES users(id) ON DELETE CASCADE,
                FOREIGN KEY(item_id) REFERENCES items(item_id) ON DELETE CASCADE
            );
        """)
        
        #  Deshabilitamos la verificación de FK temporalmente (solo para carga inicial) para evitar errores de inserción
        conn.execute("PRAGMA foreign_keys = OFF;") 

        # Insertamos los datos en las tablas correspondientes
        users_df_clean.to_sql('users', conn, if_exists='append', index=False)
        items_df_clean.to_sql('items', conn, if_exists='append', index=False) 
        preferences_df.to_sql('preferences', conn, if_exists='append', index=False)
        
        # Volvemos a habilitar la verificación de claves foráneas
        conn.execute("PRAGMA foreign_keys = ON;")
        
        conn.commit()# Guardamos los cambios
        conn.close()# Cerramos la conexión
        
        print(f" Base de datos SQLite creada y cargada con éxito.")
            
    except Exception as e:
        print(f" Error al inicializar SQLite: {e}")
        raise

def initial_load():
    """Carga los datos de SQLite (una vez que la DB existe y si no, la crea) y genera las matrices necesarias para el sistema recomendador.
    Returns:
        tuple: (df, matrix_norm, user_similarity, users_df, items_df)"""
    if not(os.path.exists(DB_NAME)): # Verificamos si la base de datos ya existe
        print(f"Base de datos '{DB_NAME}' no encontrada. Creandola e inicializándola...")
        initialize_db() #Inicializa la DB si no existe
    
    # Consulta SQL para unir items y preferences
    SQL_QUERY = """
SELECT 
    T1.item_id,
    T1.name,
    T2.user_id,
    T2.preference_value
FROM 
    items AS T1 
INNER JOIN 
    preferences AS T2
ON 
    T1.item_id = T2.item_id;
"""
    
    conn = sqlite3.connect(DB_NAME)# Abrimos la conexión a la base de datos
    df = pd.read_sql_query(SQL_QUERY, conn)# Leemos las preferencias uniendo items y preferences
    users_df = pd.read_sql_query("SELECT id, username, attributes FROM users", conn)# Leemos los usuarios desde la base de datos
    items_df = pd.read_sql_query("SELECT * FROM items", conn)# Leemos los items desde la base de datos
    conn.close()# Cerramos la conexión a la base de datos

    # Deserialización de los atributos del usuario
    if 'attributes' in users_df.columns:
        users_df['attributes'] = users_df['attributes'].apply(
            lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else {}
        )
    # Deserialización de los atributos del item
    if 'attributes' in items_df.columns:
        # 1. Deserializar el JSON y convertirlo en una Serie de diccionarios
        items_df['attributes_dict'] = items_df['attributes'].apply(
            lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else {}
        )
        # Expenimos los diccionarios en columnas separadas
        temp_attr_df = items_df['attributes_dict'].apply(pd.Series)
        # Concatenamos las columnas expandidas al DataFrame principal y renombrar para evitar colisión
        # El DataFrame ITEMS_DF final tendrá 'item_id', 'name', 'attributes' (el JSON original), y las columnas expandidas.
        # Ahora el DF contiene todos los atributos como columnas separadas.
        items_df = pd.concat([items_df, temp_attr_df], axis=1)
    if 'id' in users_df.columns: #nos aseguramos que el id sea int
        users_df['id'] = users_df['id'].astype(int)

    #Creamos las matrices necesarias para el sistema recomendador
    matrix = df.pivot_table(index='user_id', columns='name', values='preference_value')# Creamos la matriz usuario-item
    user_item_matrix = matrix.copy()# hacemos una copia para trabajar sin alterar la original
    row_mean = user_item_matrix.mean(axis=1)# Calculamos el promedio por fila
    row_std = user_item_matrix.std(axis=1)# Calculamos la desviación estándar por fila
    row_std[row_std == 0] = 1 # Evitamos división por cero
    matrix_norm = user_item_matrix.sub(row_mean, axis=0).div(row_std, axis=0)# Normalizamos la matriz usuario-item
    user_item_matrix_filled = matrix_norm.fillna(0)# Rellenamos los valores NaN con 0 para calcular similitudes
    user_similarity_cosine = cosine_similarity(user_item_matrix_filled)# Calculamos la similitud del coseno entre los usuarios
    user_ids = user_item_matrix_filled.index.tolist()# Obtenemos los IDs de usuario
    user_similarity = pd.DataFrame( #Creamos la matriz de similitud de usuarios
        user_similarity_cosine, 
        index=user_ids, 
        columns=user_ids
    )
    user_similarity.index = user_similarity.index.astype(str)# Aseguramos que los índices sean strings
    user_similarity.columns = user_similarity.columns.astype(str)# Aseguramos que las columnas sean strings
    # Retornamos los datos procesados
    return df, matrix_norm, user_similarity, users_df, items_df,row_mean

try:
    DF, MATRIX_NORM, USER_SIMILARITY, USERS_DF, ITEMS_DF, row_mean = initial_load()#
except Exception as e:# Manejo de errores en la carga inicial
    print(f" ERROR FATAL: La aplicación no pudo iniciar debido al error de carga/procesamiento: {e}")
def user_has_preferences(user_id: int) -> bool:
    """Verifica si un usuario tiene preferencias registradas en la matriz normalizada.
    Args:a
        user_id (int): ID del usuario a verificar.
    Returns:
        bool: True si el usuario tiene preferencias, False en caso contrario."""
    return user_id in MATRIX_NORM.index

def cold_start_items_recommendations(number_max_of_recommendations: int) -> list: 
    """Genera recomendaciones para usuarios nuevos basadas en los items más populares.
    Args:
        number_max_of_recommendations (int): Número máximo de items a recomendar.
    Returns: 
        list: Lista de nombres de items recomendados."""
    top_items = DF.groupby('name')['preference_value'].count().sort_values(ascending=False)# Contamos la cantidad de preferencias por item y los ordenamos
    return top_items.head(number_max_of_recommendations).index.tolist()# Retornamos los nombres de los items más populares

def get_recommendations(user_id: int, number_max_of_recommendations: int) -> list:
    """Genera recomendaciones para un usuario específico, ya sea basado en usuarios similares o en los items más populares si el usuario es nuevo.
    Args:
        user_id (int): ID del usuario para el cual se generan recomendaciones.
        number_max_of_recommendations (int): Número máximo de items a recomendar.
    Returns:
        list: Lista de nombres de items recomendados.
    """
    K_NEIGHBORS = 50 # Número usuarios similares a considerar
    if user_has_preferences(user_id): # Usuario existente con preferencias
        user_id_str = str(user_id)
        user_items = MATRIX_NORM.loc[user_id]
        items_comprados = user_items[user_items.notna()].index.tolist()

        # 1. OBTENCIÓN DE LA MEDIA HISTÓRICA DEL USUARIO (Corrección de la cancelación de scores)
        # Calculamos la media del usuario objetivo a partir del DataFrame de preferencias (DF).
        user_mean_rating = DF[DF['user_id'] == user_id]['preference_value'].mean()
        if pd.isna(user_mean_rating):
             # Fallback: si no se encuentra, usamos el valor neutro 3.0 (o la media global)
            user_mean_rating = 3.0 
        
        similar_users = (
            USER_SIMILARITY[user_id_str]
            .sort_values(ascending=False)
            .drop(user_id_str, errors='ignore')
            .head(K_NEIGHBORS)
        )
        similar_users_ids = similar_users.index.astype(int).tolist()
        sum_similarity = similar_users.sum()
        
        if similar_users.empty or sum_similarity == 0:
            return cold_start_items_recommendations(number_max_of_recommendations)
        
        # ... (Resto del código para filtrar items comprados y obtener candidatos) ...
        
        similar_user_preferences = MATRIX_NORM.loc[similar_users_ids]
        items_comprados_por_similares = similar_user_preferences.columns[
            similar_user_preferences.notna().any()
        ].tolist()
        candidate_items = list(set(items_comprados_por_similares) - set(items_comprados))
        
        if not candidate_items:
            return cold_start_items_recommendations(number_max_of_recommendations)
        
        candidate_matrix = similar_user_preferences[candidate_items].copy()
        
        # 2. CÁLCULO DE LA PREDICCIÓN AJUSTADA (La lógica crítica)
        weighted_scores = candidate_matrix.multiply(similar_users, axis=0)
        
        # Numerador: Suma ponderada de las desviaciones (scores normalizados)
        numerator = weighted_scores.fillna(0).sum(axis=0)

        # Denominador: Suma de Similitudes
        # sum_similarity ya está calculado.
        
        # Fórmula corregida: Suma Ponderada Ajustada (Deviation + User Mean)
        deviation = numerator / sum_similarity
        recommendation_scores = user_mean_rating + deviation 
        
        # 3. Post-procesamiento
        # Aseguramos que las puntuaciones estén dentro del rango de calificación (1 a 5)
        recommendation_scores = recommendation_scores.clip(lower=1.0, upper=5.0) 

        recomendaciones_ordenadas = recommendation_scores.sort_values(ascending=False)
        return recomendaciones_ordenadas.head(number_max_of_recommendations).index.tolist()
        
    else:
        # Usuario nuevo sin preferencias
        return cold_start_items_recommendations(number_max_of_recommendations)
    
# Ejemplo de uso de la función de recomendaciones
user_id = 350
get_recommendations(user_id, 10)

['La Vida en Marte',
 'Guía de Inversión Inteligente',
 'Yoga Aéreo',
 'La Balada del Viajero',
 'Manual de Liderazgo Ágil',
 'Recetas de la Abuela',
 'Microeconomía Aplicada',
 'La Conquista del Espacio',
 'Repostería Francesa',
 'Desarrollo de Videojuegos Unity']

Ejemplo de uso de la función de recomendaciones

In [281]:
user_id = 350
recommendations = get_recommendations(user_id, 10)
print(f"Items recomendados para el usuario {user_id}: {recommendations}")

Items recomendados para el usuario 350: ['La Vida en Marte', 'Guía de Inversión Inteligente', 'Yoga Aéreo', 'La Balada del Viajero', 'Manual de Liderazgo Ágil', 'Recetas de la Abuela', 'Microeconomía Aplicada', 'La Conquista del Espacio', 'Repostería Francesa', 'Desarrollo de Videojuegos Unity']


### Estrategia de Evaluación del modelo
Levaremos a cabo los siguientes pasos:

1. Separar el DataFrame de preferencias en conjuntos de entrenamiento y prueba.

2. Recalcular las matrices solo con los datos de entrenamiento.

3. Definimos una función de evaluación que itere sobre el conjunto de prueba, use nuestro sistema de recomendación para generar predicciones y calcule las métricas de las mismas.

#### 1.Separación del DataFrame de preferencias en conjuntos de entrenamiento y prueba.
Para dividir el Dataframe de preferencias en train y test usamos el método holdout, que selecciona las últimas k interacciones de cada usuario para el conjunto de prueba. 
Seleccionamos este método porque es simple y efectivo para evaluar sistemas de recomendación, debido a que refleja escenarios del mundo real donde queremos predecir futuras interacciones basadas en el historial pasado. 

In [256]:
def split_dataframe(df_full, k_holdout)-> tuple:
    """Divide el DataFrame completo en conjuntos de entrenamiento y prueba utilizando el método holdout. Este método selecciona las últimas k interacciones de cada usuario para el conjunto de prueba.
    Args:
        df_full (pd.DataFrame): DataFrame completo con las interacciones de usuario-item.
        k_holdout (int): Número de interacciones por usuario a incluir en el conjunto de prueba.
    Returns:
        tuple: (train, test) donde 'train' es el conjunto de entrenamiento y 'test' es el conjunto de prueba.
    """
   #también podríamos usar sklearn.model_selection.GroupShuffleSplit para hacer el split por usuario 
    
    test = df_full.groupby('user_id').tail(k_holdout).copy()
    train = df_full.drop(test.index).copy()
    return train, test

#### 2.Recalculamos las matrices solo con los datos de entrenamiento.
Recalculamos las matrices usuario-item y de similitud solo con los datos de entrenamiento, y preparamos el conjunto de prueba. 
Esto es necesario para evaluar el sistema recomendador, ya que no podemos usar datos de prueba para generar recomendaciones.


In [257]:
def prepare_matrices_for_evaluation(df_full, k_holdout):
    """
    calcula las matrices solo con los datos de entrenamiento y prepara el conjunto de prueba.
    """
    
    # Separamos el DataFrame en conjuntos de entrenamiento y prueba usando la función definida antes
    train_df, test_set = split_dataframe(df_full, k_holdout)
    # Contrucción de la matriz usuario-item solo con los datos de entrenamiento
    matrix = train_df.pivot_table(index='user_id', columns='name', values='preference_value')
    
    # Normalizamos la matriz
    user_item_matrix = matrix.copy()
    row_mean = user_item_matrix.mean(axis=1)
    row_std = user_item_matrix.std(axis=1)
    row_std[row_std == 0] = 1 
    matrix_norm_train = user_item_matrix.sub(row_mean, axis=0).div(row_std, axis=0)
    
    # Calculamos la similitud entre usuarios en el conjunto de entrenamiento
    user_item_matrix_filled = matrix_norm_train.fillna(0)
    user_similarity_cosine = cosine_similarity(user_item_matrix_filled)
    user_ids = user_item_matrix_filled.index.tolist()
    user_similarity_train = pd.DataFrame(
        user_similarity_cosine, 
        index=user_ids, 
        columns=user_ids
    )
    user_similarity_train.index = user_similarity_train.index.astype(str)
    user_similarity_train.columns = user_similarity_train.columns.astype(str)
    
    # Preparamos el conjunto de prueba en formato adecuado
    actual_items_test = (
        test_set
        .groupby('user_id')['name'] # Usamos el nombre del ítem, no el ID
        .apply(set)
    )
    
    return matrix_norm_train, user_similarity_train, actual_items_test, train_df, row_mean

#### 3. Función de evaluación
##### Metrica
Para evaluar el sistema recomendador primero es necesario elegir una métrica de evaluación, nosotros usaremos Recall. 
Consideramos que esta metrica es la mas adecuada ya que mide la capacidad del sistema para recuperar items relevantes para el usuario.
La interpretación de esta metrica es:
De todos los ítems que al usuario realmente compró, ¿qué porcentaje de ellos fue capturado por mi lista de recomendación.
No usamos precision porque tiene como desventaja que penaliza las recomendaciones correctas si hay muchas recomendaciones incorrectas, ya que divide el número de aciertos entre el total de recomendaciones hechas.


In [258]:


def calculate_metrics(recommended_items, actual_items,n):
    """ Calcula precision, Recall y F1 Score para un usuario dado sus items recomendados y los items reales.
    Args:
        recommended_items (list): Lista de items recomendados.
        actual_items (set): Conjunto de items reales
        n (int): numero de recomendaciones"""
    hit_items = set(recommended_items).intersection(actual_items)
    num_hits = len(hit_items)
    
    precision = num_hits / n if n > 0 else 0
    
    if len(actual_items) > 0:
        recall = num_hits / len(actual_items)
    else:
        recall = 0
        
    if (precision + recall) > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0
        
    return {
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    }

Ahora si podemos formar la funcion de evaluacion

In [259]:

def evaluate_model( n_recs=10, k_holdout=1):
  """Evalúa el sistema recomendador usando un enfoque Holdout.
  Args:
    n_recs (int): Número de recomendaciones a generar por usuario.
    k_holdout (int): Número de interacciones a reservar para el conjunto de prueba por usuario.
  """
  
  #En el programa original usabamos variables globales, ya que usamos siempre los mismos datos, aqui debemos reasignarlas localmente para la evaluación
  # Las variables globales se reasignan localmente para la prueba
  global MATRIX_NORM, USER_SIMILARITY, DF
  # Guardamos una copia de las matrices originales
  original_matrix_norm = MATRIX_NORM
  original_user_similarity = USER_SIMILARITY
  original_df = DF
  
  # Preparamos las matrices de entrenamiento y el set de prueba
  MATRIX_NORM, USER_SIMILARITY, actual_items_test, DF, ROW_MEAN_TRAIN = prepare_matrices_for_evaluation(original_df, k_holdout)

  evaluation_results = []
  
  for user_id, actual_set in actual_items_test.items():
    # Verificamos que el usuario pueda ser evaluado (tiene ítems en el test set)
    if len(actual_set) == 0:
      continue  
    # Ahora que sabemos que el usuario puede ser evaluado
    # Generamos recomendaciones usando las matrices de entrenamiento
    recommended_item_names = get_recommendations(user_id, n_recs)
    
    # Calculamos las metricas
    metrics = calculate_metrics(recommended_item_names, actual_set,n_recs)
    metrics['user_id'] = user_id
    evaluation_results.append(metrics)
    
  # Restablecemos las matrices originales
  MATRIX_NORM = original_matrix_norm
  USER_SIMILARITY = original_user_similarity
  DF = original_df
  
  # Resultado de la evaluación
  results_df = pd.DataFrame(evaluation_results)
  
  print(f"--- Resultados Promedio de la Evaluación para n={n_recs} y K-Holdout={k_holdout} ---")
  print(f"Precision: {float(results_df['Precision'].mean())}") 
  print(f"Recall: {float(results_df['Recall'].mean())}") 
  print(f"F1-Score: {float(results_df['F1-Score'].mean())}") 

### Evaluación del modelo

#### Prueba A
Planteamos un escenario similar al establecido por defecto en la especificado en la documentacion de la API y un holdout moderado.
Oculta dos interacciones recientes.
n=5
k=2

In [260]:
evaluate_model(5, 2)


--- Resultados Promedio de la Evaluación para n=5 y K-Holdout=2 ---
Precision: 0.019714285714285715
Recall: 0.04928571428571429
F1-Score: 0.028163265306122457


#### Prueba B
Esta prueba se enfoca en asegurar que las primeras recomendaciones que ve el usuario son dealta calidad, utilizando una lista muy corta.
n=3
k=4


In [ ]:
evaluate_model(3, 4)


--- Resultados Promedio de la Evaluación para n=3 y K-Holdout=4 ---
Precision: 0.06047619047619047
Recall: 0.04535714285714286
F1-Score: 0.05183673469387756


#### Prueba C
Esta prueba mide el potencial máximo de su modelo para encontrar ítems relevantes.
Utilizamos un N mucho mayor que K
n=15
k=3

In [263]:
evaluate_model(10, 3)

--- Resultados Promedio de la Evaluación para n=10 y K-Holdout=3 ---
Precision: 0.052571428571428575
Recall: 0.17523809523809522
F1-Score: 0.08087912087912087


In [276]:

# --- CONSTANTES GLOBALES (Necesitas definir DB_NAME, USERS_URL, ITEMS_URL, PREFERENCES_URL, y split_dataframe) ---
# DB_NAME, USERS_URL, ITEMS_URL, PREFERENCES_URL deben estar definidos fuera de este bloque.
# K_NEIGHBORS es el valor óptimo que encontramos en las pruebas.
K_NEIGHBORS = 10

# Declaración de variables globales que serán asignadas en initial_load()
DF = None
MATRIX_NORM = None
ITEM_SIMILARITY = None # <-- CAMBIO: De USER_SIMILARITY a ITEM_SIMILARITY
USERS_DF = None
ITEMS_DF = None
row_mean = None 
# ------------------------------------------------------------------------------------------------------------------

def initialize_db():
    """
    Crea y puebla la base de datos SOLO si el archivo DB no existe.
    Establece las claves primarias (compuestas) y foráneas.
    """
    # ... (Cuerpo de la función initialize_db sin cambios) ...
    try:
        # Cargamos los datos desde los CSV
        users_df = pd.read_csv(USERS_URL)
        items_df = pd.read_csv(ITEMS_URL)
        preferences_df = pd.read_csv(PREFERENCES_URL)
        
        # Preprocesamos los atributos de USERS
        if 'id' not in users_df.columns: 
            id_cols = [col for col in users_df.columns if 'id' in col.lower()]
            if id_cols:
                users_df = users_df.rename(columns={id_cols[0]: 'id'})
        
        BASE_KEYS = ['id', 'username']
        # Función para serializar atributos adicionales a JSON
        def serialize_attributes(row):
            attributes = {
                k: v for k, v in row.items() 
                if k not in BASE_KEYS and pd.notna(v)
            }
            return json.dumps(attributes)# Serializamos a JSON
        
        
        users_df['attributes'] = users_df.apply(serialize_attributes, axis=1)# Creamos la columna 'attributes' con JSON
        users_df_clean = users_df[BASE_KEYS + ['attributes']].copy()# Filtramos solo las columnas necesarias         
        
        # Preprocesamiento de ITEMS 
        if 'id' in items_df.columns and 'item_id' not in items_df.columns:
            items_df.rename(columns={'id': 'item_id'}, inplace=True)
            
        BASE_ITEM_KEYS = ['item_id', 'name']
        def serialize_item_attributes(row):
            attributes = {
                k: v for k, v in row.items() 
                if k not in BASE_ITEM_KEYS and pd.notna(v)
            }
            return json.dumps(attributes)
        
        items_df['attributes'] = items_df.apply(serialize_item_attributes, axis=1)
        items_df_clean = items_df[BASE_ITEM_KEYS + ['attributes']].copy()

        # Mapeo y Filtrado de PREFERENCES 
        # Aseguramos que las columnas tengan los nombres correctos
        preferences_df.rename(columns={
            'user_id': 'user_id',
            'item_id': 'item_id',
            'preference_value': 'preference_value'
        }, inplace=True)
        
        # Filtramos solo las columnas necesarias 
        preferences_df = preferences_df[['user_id', 'item_id', 'preference_value']].copy()
        
        # Aseguramos que ITEMS tenga 'item_id'
        # Asumiendo que el CSV de Items usa 'item_id'
        if 'id' in items_df.columns and 'item_id' not in items_df.columns:
            items_df.rename(columns={'id': 'item_id'}, inplace=True)
            
        # ------------------------------------------------------------------------------------------------
        # Creación de la base de datos SQLite y tablas con claves primarias y foráneas
        # ------------------------------------------------------------------------------------------------
        conn = sqlite3.connect(DB_NAME)# Conexión a la base de datos SQLite
        cursor = conn.cursor()# Cursor para ejecutar comandos SQL
        
       #Creamos tabla de usuarios con id como PRIMARY KEY y username único
        cursor.execute("""
             CREATE TABLE IF NOT EXISTS users (
                 id INTEGER PRIMARY KEY NOT NULL,
                 username TEXT UNIQUE NOT NULL,
                 attributes TEXT
             );
        """)
        
        # Creamos la tabla de items con item_id como PRIMARY KEY
        cursor.execute("""
                 CREATE TABLE IF NOT EXISTS items (
                     item_id INTEGER PRIMARY KEY NOT NULL,
                     name TEXT NOT NULL,
                     attributes TEXT
                 );
             """)
        
        #Creamos la tabla de preferencias con claves foráneas a las tablas users e items y clave primaria compuesta
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS preferences (
                user_id INTEGER NOT NULL,
                item_id INTEGER NOT NULL,
                preference_value INTEGER NOT NULL,
                PRIMARY KEY (user_id, item_id),
                FOREIGN KEY(user_id) REFERENCES users(id) ON DELETE CASCADE,
                FOREIGN KEY(item_id) REFERENCES items(item_id) ON DELETE CASCADE
            );
        """)
        
        #  Deshabilitamos la verificación de FK temporalmente (solo para carga inicial) para evitar errores de inserción
        conn.execute("PRAGMA foreign_keys = OFF;") 

        # Insertamos los datos en las tablas correspondientes
        users_df_clean.to_sql('users', conn, if_exists='append', index=False)
        items_df_clean.to_sql('items', conn, if_exists='append', index=False) 
        preferences_df.to_sql('preferences', conn, if_exists='append', index=False)
        
        # Volvemos a habilitar la verificación de claves foráneas
        conn.execute("PRAGMA foreign_keys = ON;")
        
        conn.commit()# Guardamos los cambios
        conn.close()# Cerramos la conexión
        
        print(f" Base de datos SQLite creada y cargada con éxito.")
            
    except Exception as e:
        print(f" Error al inicializar SQLite: {e}")
        raise


def initial_load():
    """Carga los datos de SQLite y genera las matrices necesarias para el sistema recomendador IBCF.
    Returns:
        tuple: (df, matrix_norm, item_similarity, users_df, items_df, row_mean)"""
    if not(os.path.exists(DB_NAME)): # Verificamos si la base de datos ya existe
        print(f"Base de datos '{DB_NAME}' no encontrada. Creandola e inicializándola...")
        initialize_db() #Inicializa la DB si no existe
    
    # Consulta SQL para unir items y preferences
    SQL_QUERY = """
SELECT 
    T1.item_id,
    T1.name,
    T2.user_id,
    T2.preference_value
FROM 
    items AS T1 
INNER JOIN 
    preferences AS T2
ON 
    T1.item_id = T2.item_id;
"""
    
    conn = sqlite3.connect(DB_NAME)# Abrimos la conexión a la base de datos
    df = pd.read_sql_query(SQL_QUERY, conn)# Leemos las preferencias uniendo items y preferences
    users_df = pd.read_sql_query("SELECT id, username, attributes FROM users", conn)# Leemos los usuarios desde la base de datos
    items_df = pd.read_sql_query("SELECT * FROM items", conn)# Leemos los items desde la base de datos
    conn.close()# Cerramos la conexión a la base de datos

    # ... (Deserialización de atributos sin cambios) ...
    if 'attributes' in users_df.columns:
        users_df['attributes'] = users_df['attributes'].apply(
            lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else {}
        )
    if 'attributes' in items_df.columns:
        items_df['attributes_dict'] = items_df['attributes'].apply(
            lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else {}
        )
        temp_attr_df = items_df['attributes_dict'].apply(pd.Series)
        items_df = pd.concat([items_df, temp_attr_df], axis=1)
    if 'id' in users_df.columns: 
        users_df['id'] = users_df['id'].astype(int)

    # Creamos las matrices necesarias para el sistema recomendador
    matrix = df.pivot_table(index='user_id', columns='name', values='preference_value')# Matriz usuario-item
    user_item_matrix = matrix.copy()
    row_mean = user_item_matrix.mean(axis=1)# Promedio por fila (media de usuario)
    row_std = user_item_matrix.std(axis=1)
    row_std[row_std == 0] = 1 
    matrix_norm = user_item_matrix.sub(row_mean, axis=0).div(row_std, axis=0)# Normalizamos por usuario

    # --- CAMBIO CLAVE: CÁLCULO DE SIMILITUD ENTRE ÍTEMS ---
    item_item_matrix_filled = matrix_norm.fillna(0).T # Transponemos la matriz normalizada
    item_similarity_cosine = cosine_similarity(item_item_matrix_filled) # Calculamos similitud entre ítems
    item_names = item_item_matrix_filled.index.tolist()
    item_similarity = pd.DataFrame( # Creamos la matriz de similitud de ítems
        item_similarity_cosine, 
        index=item_names, 
        columns=item_names
    )
    item_similarity.index = item_similarity.index.astype(str)
    item_similarity.columns = item_similarity.columns.astype(str)
    # -----------------------------------------------------

    # Retornamos los datos procesados (USER_SIMILARITY reemplazado por ITEM_SIMILARITY)
    return df, matrix_norm, item_similarity, users_df, items_df,row_mean

try:
    # --- CAMBIO DE ASIGNACIÓN GLOBAL ---
    DF, MATRIX_NORM, ITEM_SIMILARITY, USERS_DF, ITEMS_DF, row_mean = initial_load()
    # ----------------------------------
except Exception as e:
    print(f" ERROR FATAL: La aplicación no pudo iniciar debido al error de carga/procesamiento: {e}")


def user_has_preferences(user_id: int) -> bool:
    """Verifica si un usuario tiene preferencias registradas en la matriz normalizada."""
    return user_id in MATRIX_NORM.index

def cold_start_items_recommendations(number_max_of_recommendations: int) -> list: 
    """Genera recomendaciones para usuarios nuevos basadas en los items más populares."""
    top_items = DF.groupby('name')['preference_value'].count().sort_values(ascending=False)
    return top_items.head(number_max_of_recommendations).index.tolist()

def get_recommendations(user_id: int, number_max_of_recommendations: int) -> list:
    """Genera recomendaciones IBCF para un usuario específico."""
    if user_has_preferences(user_id):
        # ... (cálculo de user_mean_rating y user_ratings_norm sin cambios) ...
        
        user_mean_rating = DF[DF['user_id'] == user_id]['preference_value'].mean()
        if pd.isna(user_mean_rating):
             user_mean_rating = 3.0 
        
        user_ratings_norm = MATRIX_NORM.loc[user_id].dropna()
        items_rated_by_user = user_ratings_norm.index.tolist()
        
        # --- CAMBIO CRÍTICO: FILTRADO DE ÍTEMS CANDIDATOS PARA EVITAR BLOQUEOS ---
        
        # 1. Obtener los K ítems más populares no calificados. (Filtrado por popularidad global)
        # Esto reduce drásticamente el tamaño del bucle "for"
        top_unrated_globally = DF[~DF['name'].isin(items_rated_by_user)] \
            .groupby('name')['preference_value'].count() \
            .sort_values(ascending=False).head(300).index.tolist() # Limitar a 300 candidatos
            
        candidate_items = list(set(top_unrated_globally))
        
        # --- FIN DEL CAMBIO CRÍTICO ---
        
        if not candidate_items:
            return cold_start_items_recommendations(number_max_of_recommendations)

        # ... (El resto del código del bucle 'for' sigue igual) ...
        
        recommendation_scores = {}
        for candidate_item in candidate_items:
            
            # --- Lógica de predicción IBCF ---
            # ... (cuerpo del bucle sin cambios) ...
            
            if candidate_item not in ITEM_SIMILARITY.index:
                continue
            item_similarity_vector = ITEM_SIMILARITY.loc[candidate_item]
            
            rated_items_similarity = item_similarity_vector[items_rated_by_user]
            final_ratings = user_ratings_norm[items_rated_by_user]
            
            if K_NEIGHBORS > 0:
                neighbor_data = pd.DataFrame({
                    'similarity': rated_items_similarity,
                    'rating': final_ratings
                }).dropna()
                
                top_neighbors = neighbor_data['similarity'].abs().sort_values(ascending=False).head(K_NEIGHBORS).index
                
                final_similarity = rated_items_similarity[top_neighbors]
                final_ratings = final_ratings[top_neighbors]

            numerator = (final_similarity * final_ratings).sum()
            denominator = final_similarity.abs().sum()
            
            if denominator > 0:
                deviation = numerator / denominator
                prediction = user_mean_rating + deviation
                recommendation_scores[candidate_item] = prediction
                
        # ... (Post-procesamiento sin cambios) ...
        
        recommendation_scores_series = pd.Series(recommendation_scores)
        
        if recommendation_scores_series.empty:
            return cold_start_items_recommendations(number_max_of_recommendations)
            
        recommendation_scores_series = recommendation_scores_series.clip(lower=1.0, upper=5.0) 

        recomendaciones_ordenadas = recommendation_scores_series.sort_values(ascending=False)
        return recomendaciones_ordenadas.head(number_max_of_recommendations).index.tolist()
        
    else:
        return cold_start_items_recommendations(number_max_of_recommendations)

# --- FUNCIONES DE EVALUACIÓN (Modificadas para usar ITEM_SIMILARITY) ---

def prepare_matrices_for_evaluation(df_full, k_holdout):
    """
    Calcula las matrices de entrenamiento para IBCF y prepara el conjunto de prueba.
    """
    # ... (Asumimos split_dataframe existe y funciona)
    train_df, test_set = split_dataframe(df_full, k_holdout)
    matrix = train_df.pivot_table(index='user_id', columns='name', values='preference_value')
    
    # Normalizamos la matriz
    user_item_matrix = matrix.copy()
    row_mean = user_item_matrix.mean(axis=1)
    row_std = user_item_matrix.std(axis=1)
    row_std[row_std == 0] = 1 
    matrix_norm_train = user_item_matrix.sub(row_mean, axis=0).div(row_std, axis=0)
    
    # --- CAMBIO CLAVE: CÁLCULO DE SIMILITUD ENTRE ÍTEMS (ENTRENAMIENTO) ---
    item_item_matrix_filled_train = matrix_norm_train.fillna(0).T
    item_similarity_cosine_train = cosine_similarity(item_item_matrix_filled_train)
    item_names_train = item_item_matrix_filled_train.index.tolist()
    item_similarity_train = pd.DataFrame(
        item_similarity_cosine_train, 
        index=item_names_train, 
        columns=item_names_train
    )
    item_similarity_train.index = item_similarity_train.index.astype(str)
    item_similarity_train.columns = item_similarity_train.columns.astype(str)
    # ---------------------------------------------------------------------
    
    actual_items_test = (
        test_set
        .groupby('user_id')['name']
        .apply(set)
    )
    
    # Retorna item_similarity_train
    return matrix_norm_train, item_similarity_train, actual_items_test, train_df, row_mean,

def calculate_metrics(recommended_items, actual_items,n):
    """ Calcula precision, Recall y F1 Score para un usuario dado."""
    # ... (Cuerpo de la función sin cambios) ...
    hit_items = set(recommended_items).intersection(actual_items)
    num_hits = len(hit_items)
    
    precision = num_hits / n if n > 0 else 0
    
    if len(actual_items) > 0:
        recall = num_hits / len(actual_items)
    else:
        recall = 0
        
    if (precision + recall) > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0
        
    return {
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    }
def evaluate_model( n_recs=10, k_holdout=1):
    """Evalúa el sistema recomendador usando un enfoque Holdout para IBCF."""
    
    # Las variables globales se reasignan localmente para la prueba
    global MATRIX_NORM, ITEM_SIMILARITY, DF # <-- CAMBIO: ITEM_SIMILARITY
    # Guardamos una copia de las matrices originales
    original_matrix_norm = MATRIX_NORM
    original_item_similarity = ITEM_SIMILARITY # <-- CAMBIO: ITEM_SIMILARITY
    original_df = DF
    
    # Preparamos las matrices de entrenamiento y el set de prueba
    MATRIX_NORM, ITEM_SIMILARITY, actual_items_test, DF, ROW_MEAN_TRAIN = prepare_matrices_for_evaluation(original_df, k_holdout)

    evaluation_results = []
    
    for user_id, actual_set in actual_items_test.items():
        if len(actual_set) == 0:
            continue  
        
        # Generamos recomendaciones usando las matrices de entrenamiento IBCF
        recommended_item_names = get_recommendations(user_id, n_recs)
        
        # Calculamos las metricas
        metrics = calculate_metrics(recommended_item_names, actual_set,n_recs)
        metrics['user_id'] = user_id
        evaluation_results.append(metrics)
        
    # Restablecemos las matrices originales
    MATRIX_NORM = original_matrix_norm
    ITEM_SIMILARITY = original_item_similarity # <-- CAMBIO: ITEM_SIMILARITY
    DF = original_df
    
    # Resultado de la evaluación
    results_df = pd.DataFrame(evaluation_results)
    
    print(f"--- Resultados Promedio de la Evaluación para n={n_recs} y K-Holdout={k_holdout} ---")
    print(f"Precision: {float(results_df['Precision'].mean())}") 
    print(f"Recall: {float(results_df['Recall'].mean())}") 
    print(f"F1-Score: {float(results_df['F1-Score'].mean())}")

In [271]:
evaluate_model(10, 3)

--- Resultados Promedio de la Evaluación para n=10 y K-Holdout=3 ---
Precision: 0.05785714285714286
Recall: 0.19285714285714287
F1-Score: 0.08901098901098901


In [277]:
get_recommendations(350, 10)

['Escultura Moderna',
 'Negociación y Persuasión',
 'Cuentos Breves de Otoño',
 'Cocina Mexicana Auténtica',
 'Manual de Liderazgo Ágil',
 'El Silencio del Testigo',
 'La Venganza del Inspector',
 'Algoritmos Avanzados en Java',
 'Mundo Virtual 3.0',
 'Recetas de la Abuela']